# Developing a Data Dictionary from Qualtrics API
This code can be utilized to generate a data dictionary in csv format to accompany responses exported from the Qualtrics API.


## TOC
* [Step 1: Imports, API Setup, Loading Data](#1)
* [Step 2: Compile Data Dictionary](#2)
* [Step 3: Clean Data Dictionary](#3)
* [Step 4: Save as csv](#4)

## Step 1: Imports, API Setup, Loading Data <a class="anchor" id="1"></a>

In [ ]:
#Import libraries
import pandas as pd
import numpy as np
import re
import json
import io
from html.parser import HTMLParser
from io import StringIO
from QualtricsAPI.Survey import Responses #python package that loads responses as pandas dataframe
import os
import requests

In [ ]:
#Save tokens and survey ID here -- need Qualtrics API access -- replace xxxx below with credentials and survey ID
token="xxxx"
data_center="xxxx"
directory_id="xxxx"
survey="xxxx"

#### Load survey response data from QualtricsAPI python package

In [ ]:
#Initital QualtricsAPI setup
from QualtricsAPI.Setup import Credentials

#Create an instance of Credentials
c = Credentials()

#Call the qualtrics_api_credentials() method
c.qualtrics_api_credentials(token, data_center, directory_id)

In [ ]:
#Create an instance
r = Responses()

#Load responses into df
df = r.get_survey_responses(survey)    

In [ ]:
#Will use this dict later to rename vars

#Create dictionary with each QID : Var_Name (where the var name is the Question_Export_Tag)
api_q_dict = {}
for i in df.columns:
    api_q_dict.update({json.loads(df.iloc[1][i])['ImportId']: i})

#### Load survey design from the Get Survey API (Qualtrics API)

In [ ]:
## Setting user Parameters - this will throw an error after it is set, you have to then comment out
#apiToken = os.environ[token] 
#dataCenter = os.environ[data_center]

baseUrl = "https://{0}.qualtrics.com/API/v3/surveys/{1}".format(os.environ['data_center'], survey)
headers = {
    "x-api-token": os.environ['token'],
    }

response = requests.get(baseUrl, headers=headers)
data = response.json()

#Set data to result level
data = data['result']

## Step 2: Compile Data Dictionary <a class="anchor" id="2"></a>

### Questions
* Each question type has variations in their design so have to compile the data dictionary by question type.
* Includes the following commonly used question types:
    * Multiple choice (multi & single select)
    * Text entry (general & form)
    * Matrix (text entry & non-text entry)
    * Side-by-Side (text entry & non-text entry)

#### Multi-select Multiple Choice Variables

In [ ]:
#Create list of QIDs for multi-select variables
multi_select_range = []
for i in data['questions']:
    if ((data['questions'][i]['questionType']['type'] == 'MC') & (data['questions'][i]['questionType']['selector'] == 'MAVR')):
        multi_select_range.append(i)

In [ ]:
#Create multi-select variable name list
ms_var_list = []

#Need to select values from the key, value pairs of the choices
for i in multi_select_range:
    for key, value in data['questions'][i]['choices'].items():
        #then need to add that to the data questionName to get the desired variable names
        ms_var_list.append(str(data['questions'][i]['questionName']) + '_' + str(value['recode']))

In [ ]:
#Get variable labels
ms_var_label_list = []

#Need to select values from the key, value pairs of the choices
for i in multi_select_range:
    for key, value in data['questions'][i]['choices'].items():
        #then need to add that to the data questionText to get the desired variable labels
        ms_var_label_list.append(str(data['questions'][i]['questionText']) + ' - ' + str(value['choiceText']))


In [ ]:
#Create value labels dictionary: For multi-select, value labels will be 1 'Yes', 0 'No'

#Create dictionary with response values and their corresponding labels
ms_inner_dict = {'1': 'Yes', '0': 'No'}

#Create nested dictionary w. var names : values
ms_value_label_dict = {ms_var_list[i] : ms_inner_dict for i in range(len(ms_var_list))}

#Create values df 
ms_rows = []
for key, values in ms_value_label_dict.items():
    for k, v in values.items():
        ms_rows.append([key,k,v])
ms_values_df = pd.DataFrame(ms_rows, columns=["Var_Name", "Value","Value_Label"])

In [ ]:
#Create data_dict_df
data_dict_df = pd.DataFrame(columns = ["Var_Name","Var_Label","Value","Value_Label","Data_Type"])

#Add multi-select vars to codebook if any
if len(ms_var_list) > 0:
    #first create a placeholder df with Var_Name & Var_Labels
    p_data_dict_df = pd.concat([pd.Series(x) for x in [ms_var_list, ms_var_label_list]], axis=1).rename(columns={0:"Var_Name",1:"Var_Label"})
    #merge in values df
    p_data_dict_df = pd.merge(p_data_dict_df, ms_values_df, on='Var_Name', how='inner')
    #assign data type -- INT because categorical
    p_data_dict_df['Data_Type'] = "INT"
    #assign question type
    p_data_dict_df['Question_Type'] = "multi-select"

    #merge into data_dict_df
    data_dict_df = pd.concat([data_dict_df,p_data_dict_df])

#### Multi-select Multiple Choice Other Text Entry Variables

In [ ]:
ms_ote_var_list = []
for i in multi_select_range:
    for key, value in data['questions'][i]['choices'].items():
        for key2,value2 in value.items():
            if key2 == 'textEntry':
                ms_ote_var_list.append(str(data['questions'][i]['questionName']) + '_' + str(value['recode']) + '_TEXT')


In [ ]:
#Get variable labels
ms_ote_var_label_list = []

#Need to select values from the key, value pairs of the choices
for i in multi_select_range:
    for key, value in data['questions'][i]['choices'].items():
        for key2,value2 in value.items():
            if key2 == 'textEntry':
                ms_ote_var_label_list.append(str(data['questions'][i]['questionText']) + ' - ' + str(value['choiceText'] + ' - TEXT'))



In [ ]:
#Create value labels: For mult-select other text entry, value labels will be {nan: 'text'}

#Create dictionary with responses
ms_ote_inner_dict = {np.nan: 'text'}

#Create nested dictionary w. var names : values
ms_ote_value_label_dict = {ms_ote_var_list[i] : ms_ote_inner_dict for i in range(len(ms_ote_var_list))}

#Create values df 
ms_ote_rows = []
for key, values in ms_ote_value_label_dict.items():
    for k, v in values.items():
        ms_ote_rows.append([key,k,v])
ms_ote_values_df = pd.DataFrame(ms_ote_rows, columns=["Var_Name", "Value","Value_Label"])


In [ ]:
#Update data_dict_df with text-entry vars if any
if len(ms_ote_var_list) > 0:
    #first create a placeholder df with Var_Name & Var_Labels
    p_data_dict_df = pd.concat([pd.Series(x) for x in [ms_ote_var_list, ms_ote_var_label_list]], axis=1).rename(columns={0:"Var_Name",1:"Var_Label"})
    #merge in values df
    p_data_dict_df = pd.merge(p_data_dict_df, ms_ote_values_df, on='Var_Name', how='inner')
    #assign data type -- STRING because text entry
    p_data_dict_df['Data_Type'] = "STRING"
    #assign question type
    p_data_dict_df['Question_Type'] = "multi-select text-entry"
    #merge into data_dict_df
    data_dict_df = pd.concat([data_dict_df,p_data_dict_df])

#### Single-select MC Variables

In [ ]:
#Create list of QIDs for single select variables
single_select_range = []
for i in data['questions']:
    if ((data['questions'][i]['questionType']['type'] == 'MC') & (data['questions'][i]['questionType']['selector'] != 'MAVR')):
        single_select_range.append(i)

In [ ]:
#Create single-select variable name list
single_var_list = []

#pull from questionNames
for i in single_select_range:
    single_var_list.append(data['questions'][i]['questionName'])

In [ ]:
#Get variable labels
single_var_label_list = []

#pull from questionText
for i in single_select_range:
    single_var_label_list.append(data['questions'][i]['questionText'])

In [ ]:
#Create list of value and label dictionaries
single_value_label_dict_list = []

#Iterate through the choices
for i in single_select_range:
    single_inner_dict = {}
    for j in data['questions'][i]['choices']:
        #for each, create a dict with the recode value as keys and corresp. choiceText as values (the labels)
        single_inner_dict.update({data['questions'][i]['choices'][j]['recode'] : data['questions'][i]['choices'][j]['choiceText']})
    #add each into a list
    single_value_label_dict_list.append(single_inner_dict)

#Create nested dictionary w. each var name as the keys and each dict in the list created above as the values  
single_value_label_dict = {single_var_list[i] : single_value_label_dict_list[i] for i in range(len(single_var_list))}

#Create values df
single_rows = []
for key, values in single_value_label_dict.items():
    for k, v in values.items():
        single_rows.append([key,k,v])
single_values_df = pd.DataFrame(single_rows, columns=["Var_Name", "Value","Value_Label"])

In [ ]:
#Update data_dict_df with single-select vars if any
if len(single_var_list) > 0:
    #first create a placeholder df with Var_Name & Var_Labels
    p_data_dict_df = pd.concat([pd.Series(x) for x in [single_var_list, single_var_label_list]], axis=1).rename(columns={0:"Var_Name",1:"Var_Label"})
    #merge in values df
    p_data_dict_df = pd.merge(p_data_dict_df, single_values_df, on='Var_Name', how='inner')
    #assign data type -- INT because categorical
    p_data_dict_df['Data_Type'] = "INT"
    #assign question type
    p_data_dict_df['Question_Type'] = "single-select"    
    #merge into data_dict_df
    data_dict_df = pd.concat([data_dict_df,p_data_dict_df])

#### Single-select Multiple Choice Other Text Entry Variables

In [ ]:
#Create single-select variable name list
single_var_list = []

#pull from questionNames
for i in single_select_range:
    single_var_list.append(data['questions'][i]['questionName'])

In [ ]:
#Create var list
single_ote_var_list = []
for i in single_select_range:
    for key, value in data['questions'][i]['choices'].items():
        for key2,value2 in value.items():
            if key2 == 'textEntry':
                single_ote_var_list.append(str(data['questions'][i]['questionName']) + '_' + str(value['recode']) + '_TEXT')

In [ ]:
#Get variable labels
single_ote_var_label_list = []

#Need to select values from the key, value pairs of the choices
for i in single_select_range:
    for key, value in data['questions'][i]['choices'].items():
        for key2,value2 in value.items():
            if key2 == 'textEntry':
                single_ote_var_label_list.append(str(data['questions'][i]['questionText']) + ' - ' + str(value['choiceText'] + ' - TEXT'))


In [ ]:
#Create value labels: For mult-select other text entry, value labels will be {nan: 'text'}

#Create dictionary with responses
single_ote_inner_dict = {np.nan: 'text'}

#Create nested dictionary w. var names : values
single_ote_value_label_dict = {single_ote_var_list[i] : single_ote_inner_dict for i in range(len(single_ote_var_list))}

#Create values df 
single_ote_rows = []
for key, values in single_ote_value_label_dict.items():
    for k, v in values.items():
        single_ote_rows.append([key,k,v])
single_ote_values_df = pd.DataFrame(single_ote_rows, columns=["Var_Name", "Value","Value_Label"])

In [ ]:
#Update data_dict_df with text-entry vars if any
if len(single_ote_var_list) > 0:
    #first create a placeholder df with Var_Name & Var_Labels
    p_data_dict_df = pd.concat([pd.Series(x) for x in [single_ote_var_list, single_ote_var_label_list]], axis=1).rename(columns={0:"Var_Name",1:"Var_Label"})
    #merge in values df
    p_data_dict_df = pd.merge(p_data_dict_df, single_ote_values_df, on='Var_Name', how='inner')
    #assign data type -- STRING because text entry
    p_data_dict_df['Data_Type'] = "STRING"
    #assign question type
    p_data_dict_df['Question_Type'] = "single-select text-entry"
    #merge into data_dict_df
    data_dict_df = pd.concat([data_dict_df,p_data_dict_df])

#### Text Entry Variables - Not Form

In [ ]:
#Create list of QIDs for Text Entry variables
te_range = []
for i in data['questions']:
    if ((data['questions'][i]['questionType']['type'] == 'TE') & (data['questions'][i]['questionType']['selector'] != 'FORM')):
        te_range.append(i)

In [ ]:
#Create text entry variable name list
te_var_list = []

#pull from questionName
for i in te_range:
    te_var_list.append(data['questions'][i]['questionName'])

In [ ]:
#Get variable labels
te_var_label_list = []

#pull from questionText
for i in te_range:
    te_var_label_list.append(data['questions'][i]['questionText'])

In [ ]:
#Create value labels: For text entry, value labels will be {'': 'text'}

#Create dictionary with responses
te_inner_dict = {np.nan: 'text'}

#Create nested dictionary w. var names : values
te_value_label_dict = {te_var_list[i] : te_inner_dict for i in range(len(te_var_list))}

#Create values df 
te_rows = []
for key, values in te_value_label_dict.items():
    for k, v in values.items():
        te_rows.append([key,k,v])
te_values_df = pd.DataFrame(te_rows, columns=["Var_Name", "Value","Value_Label"])

In [ ]:
#Update data_dict_df with text-entry vars if any
if len(te_var_list) > 0:
    #first create a placeholder df with Var_Name & Var_Labels
    p_data_dict_df = pd.concat([pd.Series(x) for x in [te_var_list, te_var_label_list]], axis=1).rename(columns={0:"Var_Name",1:"Var_Label"})
    #merge in values df
    p_data_dict_df = pd.merge(p_data_dict_df, te_values_df, on='Var_Name', how='inner')
    #assign data type -- STRING because text entry
    p_data_dict_df['Data_Type'] = "STRING"
    #assign question type
    p_data_dict_df['Question_Type'] = "text-entry"
    #merge into data_dict_df
    data_dict_df = pd.concat([data_dict_df,p_data_dict_df])

#### Text Entry Variables - Form

In [ ]:
#Create list of QIDs for Text Entry - Form variables
te_form_range = []
for i in data['questions']:
    if ((data['questions'][i]['questionType']['type'] == 'TE') & (data['questions'][i]['questionType']['selector'] == 'FORM')):
        te_form_range.append(i)

In [ ]:
#Create TE form variable name list
te_form_var_list = []

#Need to select key from the key, value pairs of the choices
for i in te_form_range:
    for key, value in data['questions'][i]['choices'].items():
        #then need to add that to the data questionName to get the desired variable names
        te_form_var_list.append(str(data['questions'][i]['questionName']) + '_' + str(key))

In [ ]:
#Get variable labels
te_form_var_label_list = []

#Need to select value descriptions from the key, value pairs of the choices
for i in te_form_range:
    for key, value in data['questions'][i]['choices'].items():
        #then need to add that to the data questionText to get the desired variable labels
        te_form_var_label_list.append(str(data['questions'][i]['questionText']) + ' - ' + str(value['description']))

In [ ]:
#Create dictionary with responses
te_form_inner_dict = {np.nan: 'text'}

#Create nested dictionary w. var names : values
te_form_value_label_dict = {te_form_var_list[i] : te_form_inner_dict for i in range(len(te_form_var_list))}

#Create values df 
te_form_rows = []
for key, values in te_form_value_label_dict.items():
    for k, v in values.items():
        te_form_rows.append([key,k,v])
te_form_values_df = pd.DataFrame(te_form_rows, columns=["Var_Name", "Value","Value_Label"])

In [ ]:
#Update data_dict_df with text entry vars if any
if len(te_form_var_list) > 0:
    #first create a placeholder df with Var_Name & Var_Labels
    p_data_dict_df = pd.concat([pd.Series(x) for x in [te_form_var_list, te_form_var_label_list]], axis=1).rename(columns={0:"Var_Name",1:"Var_Label"})
    #merge in values df
    p_data_dict_df = pd.merge(p_data_dict_df, te_form_values_df, on='Var_Name', how='inner')
    #assign data type -- STRING for text entry
    p_data_dict_df['Data_Type'] = "STRING"
    #assign question type
    p_data_dict_df['Question_Type'] = "text-entry form"
    #merge into data_dict_df
    data_dict_df = pd.concat([data_dict_df,p_data_dict_df])

#### Matrix Variables - Not TE
* For Matrix Variables where the selector is not TE

In [ ]:
#Create list of QIDs for matrix (non-text entry) variables
matrix_range = []
for i in data['questions']:
    if ((data['questions'][i]['questionType']['type'] == 'Matrix') & (data['questions'][i]['questionType']['selector'] != 'TE')):
        matrix_range.append(i)

In [ ]:
#Create matrix variable name list
matrix_var_list = []

#Need to select value -- recode value from the key, value pairs of the subQuestions (statements/rows of the matrix)
for i in matrix_range:
    for key, value in data['questions'][i]['subQuestions'].items():
        #then need to add that to the QID to get variable importid
        matrix_var_list.append(str(i) + '_' + str(value['recode']))

#Then rename the importid with the question export tag from api_q_dict created above
matrix_var_list = [api_q_dict.get(item,item) for item in matrix_var_list] 

In [ ]:
#Get variable labels
matrix_var_label_list = []

#Need to select value -- choiceText from the key, value pairs of the subQuestions (statements/rows of the matrix)
for i in matrix_range:
    for key, value in data['questions'][i]['subQuestions'].items():
        #then need to add that to the data questionText to get the desired variable labels
        matrix_var_label_list.append(str(data['questions'][i]['questionText']) + ' - ' + str(value['choiceText']))

In [ ]:
#Create list of value and label dictionaries
matrix_value_label_dict_list = []

#Iterate through the questions and then the choices
for i in matrix_range:
    matrix_inner_dict = {}
    for k in data['questions'][i]['subQuestions'].items():
        for j in data['questions'][i]['choices']:
            #for each, create a dict with the recode value as keys and corresp. choiceText as values (the labels)
            matrix_inner_dict.update({data['questions'][i]['choices'][j]['recode'] : data['questions'][i]['choices'][j]['choiceText']})
        #add each into a list
        matrix_value_label_dict_list.append(matrix_inner_dict)


#Create nested dictionary w. var names : {the dict created above}  
matrix_value_label_dict = {matrix_var_list[i] : matrix_value_label_dict_list[i] for i in range(len(matrix_var_list))}

#Create values df
matrix_rows = []
for key, values in matrix_value_label_dict.items():
    for k, v in values.items():
        matrix_rows.append([key,k,v])
matrix_values_df = pd.DataFrame(matrix_rows, columns=["Var_Name", "Value","Value_Label"])

In [ ]:
#Update data_dict_df with matrix vars if any
if len(matrix_var_list) > 0:
    #first create a placeholder df with Var_Name & Var_Labels
    p_data_dict_df = pd.concat([pd.Series(x) for x in [matrix_var_list, matrix_var_label_list]], axis=1).rename(columns={0:"Var_Name",1:"Var_Label"})
    #merge in values df
    p_data_dict_df = pd.merge(p_data_dict_df, matrix_values_df, on='Var_Name', how='inner')
    #assign data type -- INT because categorical
    p_data_dict_df['Data_Type'] = "INT"
    #assign question type
    p_data_dict_df['Question_Type'] = "matrix"
    #merge into data_dict_df
    data_dict_df = pd.concat([data_dict_df,p_data_dict_df])

#### Matrix Variables - Text Entry
* For Matrix Variables where the selector is TE (text entry)
* No value labels

In [ ]:
#Create list of QIDs for matrix te variables
matrix_te_range = []
for i in data['questions']:
    if ((data['questions'][i]['questionType']['type'] == 'Matrix') & (data['questions'][i]['questionType']['selector'] == 'TE')):
        matrix_te_range.append(i)

In [ ]:
#Create matrix variable name list
matrix_te_var_list = []

#Need to select value -- recode value from the key, value pairs of the subQuestions (statements/matrix rows)
for i in matrix_te_range:
    for key, value in data['questions'][i]['subQuestions'].items():
        #and select the key from the key, value pairs of the choices
        for key2, value2 in data['questions'][i]['choices'].items():
            #then need to add each of those to the QID to get variable importid
            matrix_te_var_list.append(str(i) + '_' + str(value['recode']) + "_" + str(key2))

#Then rename the importid with the question export tag from api_q_dict created above
matrix_te_var_list = [api_q_dict.get(item,item) for item in matrix_te_var_list] 

In [ ]:
#Get variable labels
matrix_te_var_label_list = []

#Need to select value -- choiceText from the key, value pairs of the subQuestions
for i in matrix_te_range:
    for key, value in data['questions'][i]['subQuestions'].items():
        #and select the value -- choiceText from the key, value pairs of the choices
        for key2, value2 in data['questions'][i]['choices'].items():
            #then need to add both of those to the questionText to get the desired variable labels
            matrix_te_var_label_list.append(str(data['questions'][i]['questionText']) + ' - ' + str(value['choiceText'])+ ' - ' + str(value2['choiceText']))


In [ ]:
#Create dictionary with responses, nan : 'text' because text entry
matrix_te_inner_dict = {np.nan: 'text'}

#Create nested dictionary w. var names : values
matrix_te_value_label_dict = {matrix_te_var_list[i] : matrix_te_inner_dict for i in range(len(matrix_te_var_list))}

#Create values df 
matrix_te_rows = []
for key, values in matrix_te_value_label_dict.items():
    for k, v in values.items():
        matrix_te_rows.append([key,k,v])
matrix_te_values_df = pd.DataFrame(matrix_te_rows, columns=["Var_Name", "Value","Value_Label"])


In [ ]:
#Update data_dict_df with matrix text entry vars if any
if len(matrix_te_var_list) > 0:
    #first create a placeholder df with Var_Name & Var_Labels
    p_data_dict_df = pd.concat([pd.Series(x) for x in [matrix_te_var_list, matrix_te_var_label_list]], axis=1).rename(columns={0:"Var_Name",1:"Var_Label"})
    #merge in values df
    p_data_dict_df = pd.merge(p_data_dict_df, matrix_te_values_df, on='Var_Name', how='inner')
    #assign data type -- STRING because text entry (may need to edit later)
    p_data_dict_df['Data_Type'] = "STRING"
    #assign question type
    p_data_dict_df['Question_Type'] = "matrix"
    #merge into data_dict_df
    data_dict_df = pd.concat([data_dict_df,p_data_dict_df])

#### SBS (Side by Side) Variables - Not TE
* For SBS Variables where the selector is not TE

In [ ]:
#Create list of QIDs for all sbs vars
sbs_range = []
for i in data['questions']:
    if (data['questions'][i]['questionType']['type'] == 'SBS'):
        sbs_range.append(i)

#Select != 'TE' from columns -- question type
sbs_nte_range = []
for i in sbs_range:
     for key, value in data['questions'][i]['columns'].items(): 
        if (data['questions'][i]['columns'][key]['questionType']['selector'] != 'TE'):
            sbs_nte_range.append(i)

#Remove duplicates
sbs_nte_range_dup = []
[sbs_nte_range_dup.append(i) for i in sbs_nte_range if i not in sbs_nte_range_dup]
sbs_nte_range = sbs_nte_range_dup

In [ ]:
#Create side by side non-text entry variable name list
sbs_nte_var_list = []

#Need to select key from the key, value pairs of the columns
for i in sbs_nte_range:
    for key, value in data['questions'][i]['columns'].items():
        #Select only non-text entry columns
        if (data['questions'][i]['columns'][key]['questionType']['selector'] != 'TE'):
            #and select key from the key, value pairs of the subQuestions
            for key2, value2 in data['questions'][i]['subQuestions'].items():
                #then need to add both of those to the QID to get variable importid
                sbs_nte_var_list.append(str(i) + '#' + str(key) + "_" + str(key2))

#Then rename the importid with the question export tag from api_q_dict created above
sbs_nte_var_list = [api_q_dict.get(item,item) for item in sbs_nte_var_list]    

In [ ]:
#Get variable labels
sbs_nte_var_label_list = []

#Need to select value -- questionText from the key, value pairs of the columns
for i in sbs_nte_range:
    for key, value in data['questions'][i]['columns'].items():
        #Select only non-text entry columns
        if (data['questions'][i]['columns'][key]['questionType']['selector'] != 'TE'):
            #and select value -- choiceText from the key, value pairs of the subQuestions (statements/rows)
            for key2, value2 in data['questions'][i]['subQuestions'].items():
                #then need to concatenate those to get the desired variable label
                sbs_nte_var_label_list.append(str(value['questionText']) + ' - ' + str(value2['choiceText']))
  

In [ ]:
# Create values list
sbs_nte_value_label_dict_list = []

for i in sbs_nte_range:
    # Iterate through the key, value pairs of the columns
    for key, value in data['questions'][i]['columns'].items():
        #Select only non-text entry columns
        if (data['questions'][i]['columns'][key]['questionType']['selector'] != 'TE'):
            #Create an empty dictionary
            sbs_nte_inner_dict = {}
            # Then iterate through value -- choices for each column
            for key2, value2 in value['choices'].items():
                # Add the recode value and its corresponding label to the dictionary
                sbs_nte_inner_dict.update({value2['recode'] : value2['choiceText']})
            # Then add each dictionary of values and labels to a list for each statement/row
            for j in range(len(data['questions'][i]['subQuestions'])): 
                sbs_nte_value_label_dict_list.append(sbs_nte_inner_dict)

#Create nested dictionary w. var names : values   
sbs_nte_value_label_dict = {sbs_nte_var_list[i] : sbs_nte_value_label_dict_list[i] for i in range(len(sbs_nte_var_list))}


In [ ]:
#Create sbs nte values df
sbs_nte_rows = []
for key, values in sbs_nte_value_label_dict.items():
    for k, v in values.items():
        sbs_nte_rows.append([key,k,v])

sbs_nte_values_df = pd.DataFrame(sbs_nte_rows, columns=["Var_Name", "Value","Value_Label"])


In [ ]:
#Update data_dict_df with side by side non-text entry vars if any
if len(sbs_nte_var_list) > 0:
    #first create a placeholder df with Var_Name & Var_Labels
    p_data_dict_df = pd.concat([pd.Series(x) for x in [sbs_nte_var_list, sbs_nte_var_label_list]], axis=1).rename(columns={0:"Var_Name",1:"Var_Label"})
    #merge in values df
    p_data_dict_df = pd.merge(p_data_dict_df, sbs_nte_values_df, on='Var_Name', how='inner')
    #assign data type -- INT because categorical
    p_data_dict_df['Data_Type'] = "INT"
    #assign question type
    p_data_dict_df['Question_Type'] = "side-by-side"
    #merge into data_dict_df
    data_dict_df = pd.concat([data_dict_df,p_data_dict_df])

#### SBS (Side by Side) Variables - Text Entry
* For side by side variables where the selector is TE

In [ ]:
#Create list of QIDs for all sbs var columns where the selector is TE
sbs_te_range = []
for i in sbs_range:
     for key, value in data['questions'][i]['columns'].items(): 
            if (data['questions'][i]['columns'][key]['questionType']['selector'] == 'TE'):
                sbs_te_range.append(i)
            
#Remove duplicates
sbs_te_range_dup = []
[sbs_te_range_dup.append(i) for i in sbs_te_range if i not in sbs_te_range_dup]
sbs_te_range = sbs_te_range_dup

In [ ]:
#Create side by side text-entry variable name list
sbs_te_var_list = []

#Need to select key from the key, value pairs of the columns
for i in sbs_te_range:
    for key, value in data['questions'][i]['columns'].items():
        #Select only text entry columns
        if (data['questions'][i]['columns'][key]['questionType']['selector'] == 'TE'):
            #and select key from key, value pairs of the subQuestions (statements/rows)
            for key2, value2 in data['questions'][i]['subQuestions'].items():
                #then add both of those and a trailing _1 to the QID to get the variable importid
                sbs_te_var_list.append(str(i) + '#' + str(key) + "_" + str(key2) + "_1")

#Then rename the importid with the question export tag from the api var list (df)
sbs_te_var_list = [api_q_dict.get(item,item) for item in sbs_te_var_list] 

In [ ]:
#Get variable labels
sbs_te_var_label_list = []

#Need to select value -- questionText from the key, value pairs of the columns
for i in sbs_te_range:
    for key, value in data['questions'][i]['columns'].items():
        #Select only text entry columns
        if (data['questions'][i]['columns'][key]['questionType']['selector'] == 'TE'):
            #and select value -- choiceText from the key, value pairs of the subQuestions (statements/rows)
            for key2, value2 in data['questions'][i]['subQuestions'].items():
                #then need to concatenate those to get the desired variable label
                sbs_te_var_label_list.append(str(value['questionText']) + ' - ' + str(value2['choiceText']))

In [ ]:
#Create dictionary with responses
sbs_te_inner_dict = {np.nan: 'text'}

#Create nested dictionary w. var names : values
sbs_te_value_label_dict = {sbs_te_var_list[i] : sbs_te_inner_dict for i in range(len(sbs_te_var_list))}

#Create values df 
sbs_te_rows = []
for key, values in sbs_te_value_label_dict.items():
    for k, v in values.items():
        sbs_te_rows.append([key,k,v])
sbs_te_values_df = pd.DataFrame(sbs_te_rows, columns=["Var_Name", "Value","Value_Label"])

In [ ]:
#Update data_dict_df with side by side text entry vars if any
if len(sbs_te_var_list) > 0:
    #first create a placeholder df with Var_Name & Var_Labels
    p_data_dict_df = pd.concat([pd.Series(x) for x in [sbs_te_var_list, sbs_te_var_label_list]], axis=1).rename(columns={0:"Var_Name",1:"Var_Label"})
    #merge in values df
    p_data_dict_df = pd.merge(p_data_dict_df, sbs_te_values_df, on='Var_Name', how='inner')
    #assign data type -- INT because categorical
    p_data_dict_df['Data_Type'] = "INT"
    #assign question type
    p_data_dict_df['Question_Type'] = "side-by-side"
    #merge into data_dict_df
    data_dict_df = pd.concat([data_dict_df,p_data_dict_df])

### Embedded Data
* Add each embedded data variable to the data dictionary
* Will add to values data_dict_df after, as there will be manual labeling needed

In [ ]:
#Create Embedded Data Vars list
ED_var_list = []

for i in range(len(data['embeddedData'])):
    ED_var_list.append(data['embeddedData'][i]['name'])

In [ ]:
#Update data_dict_df with embedded data -- just var name for now
if len(ED_var_list) > 0:
    #first create a placeholder df with Var_Name
    p_data_dict_df = pd.concat([pd.Series(x) for x in [ED_var_list]], axis=1).rename(columns={0:"Var_Name"})
    #assign question type
    p_data_dict_df['Question_Type'] = "embedded data"
    #merge into data_dict_df
    data_dict_df = pd.concat([data_dict_df,p_data_dict_df])

## Step 3: Clean Data Dictionary <a class="anchor" id="3"></a>

#### Update Qualtrics Survey Metadata Variables

In [ ]:
#Create dictionary with variable names, variable labels, values (if categorical), value labels, and data type
metadata_dict = {
    'Var_Name' : ['StartDate','EndDate','Status','Status','Status','Status','Status','Status','Status','Status',
                  'IPAddress','Progress','Duration (in seconds)','Finished','Finished','RecordedDate','ResponseId',
                  'RecipientLastName','RecipientFirstName','RecipientEmail','ExternalReference','LocationLatitude',
                  'LocationLongitude','DistributionChannel','UserLanguage'],
    'Var_Label' : ['Date and time when respondent first clicked the survey link',
                   'Date and time when respondent submitted the survey / last interacted with survey',
                   'Type of response collected','Type of response collected','Type of response collected','Type of response collected','Type of response collected','Type of response collected','Type of response collected','Type of response collected',
                   'Respondent IP address',
                   'Progress respondent made in the survey',
                   'Number of seconds it took respondent to complete the survey',
                   'Indicator for whether response was submitted','Indicator for whether response was submitted',
                   'Date and time when response was recorded in Qualtrics',
                   'Unique ID Qualtrics uses to identify each response',
                   'Recipient last name, if response from contact list distribution',
                   'Recipient first name, if response from contact list distribution',
                   'Recipient email address, if response from contact list distribution',
                   'User defined unique ID for recipient, if response from contact list distribution',
                   'Approximate latitude of location based on IPAddress',
                   'Approximate longitude of location based on IPAddress',
                   'Method of survey distribution for response',
                   'Language code of response, if survey distributed in multiple languages'],
    'Value' : [np.nan,np.nan,0,1,2,4,8,9,16,17,np.nan,np.nan,np.nan,0,1,np.nan,np.nan,np.nan,
                   np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan],
    'Value_Label' : ['date_time','date_time','IP Address','Survey Preview','Survey Test','Imported','Spam','Preview Spam','Offline','Offline Preview',
                     'text','numeric','numeric','Not submitted','Submitted','date_time','text',
                     'text','text','text','text','text','text','text','text'],
    'Data_Type' : ['TIMESTAMP','TIMESTAMP','INT','INT','INT','INT','INT','INT','INT','INT','STRING','FLOAT',
                   'INT','INT','INT','TIMESTAMP','STRING','STRING','STRING','STRING','STRING','STRING','STRING','STRING',
                   'STRING']
}

#convert to pandas df
metadata_df = pd.DataFrame(metadata_dict)

#Add Question_Type
metadata_df['Question_Type'] = 'survey metadata'

#Drop metadata variables from data_dict_df
metadata_var_list = ['StartDate','EndDate','Status','IPAddress','Progress','Duration (in seconds)','Finished',
                     'RecordedDate','ResponseId','RecipientLastName','RecipientFirstName','RecipientEmail',
                     'ExternalReference','LocationLatitude','LocationLongitude','DistributionChannel','UserLanguage']
data_dict_df = data_dict_df[~data_dict_df['Var_Name'].isin(metadata_var_list)].copy()

#Concat data_dict_df with metadata_df
data_dict_df = pd.concat([data_dict_df,metadata_df])

#### Variable order
* Get variable order from responses dataset
* Merge with data dictionary
* Sort data dictionary

In [ ]:
#Create Var_Order variable to have consistent order of variables between codebook and df

#Create list of columns
column_list = df.columns.to_list()

#Get order from columns
var_order = []
[var_order.append(i + 1) for i in range(len(column_list))]

#Create a df with columns & their order
var_order_df = pd.DataFrame({'Var_Name': column_list, 'Var_Order':var_order})

#Merge into data_dict_df - outer to see if any errors occured
data_dict_df = pd.merge(data_dict_df, var_order_df, on='Var_Name', how='outer')

In [ ]:
#Sort data_dict_df by 'Value' then 'Var_Order'
data_dict_df['Value'] = data_dict_df['Value'].apply(pd.to_numeric, errors='coerce')
data_dict_df['Var_Order'] = data_dict_df['Var_Order'].apply(pd.to_numeric)
data_dict_df = data_dict_df.sort_values(by=['Var_Order','Var_Name','Value'])

#Reorder variables
data_dict_df = data_dict_df[['Var_Name','Var_Label','Value','Value_Label','Data_Type','Var_Order','Question_Type']]

In [ ]:
#Reset the index of the data dictionary
data_dict_df = data_dict_df.reset_index().drop(columns='index')

#### Add info for embedded data variables
* Value labels and data type for categorical embedded data variables cannot be determined without manual override
* All will be set to STRING data type by default and missing values for value and value labels
* Adjust logic as needed based on survey embedded data values

In [ ]:
#Assign Var_Label from copy of Var_Name
data_dict_df['Var_Label'] = np.where(data_dict_df['Question_Type']=='embedded data', data_dict_df['Var_Name'], data_dict_df['Var_Label'])

#Assign Data_Type (STRING)
data_dict_df['Data_Type'] = np.where(data_dict_df['Question_Type']=='embedded data', 'STRING', data_dict_df['Data_Type'])

#### Clean up variable & value labels

In [ ]:
#Functions to strip html
class MLStripper(HTMLParser):
    def __init__(self):
        super().__init__()
        self.reset()
        self.strict = False
        self.convert_charrefs= True
        self.text = StringIO()
    def handle_data(self, d):
        self.text.write(d)
    def get_data(self):
        return self.text.getvalue()

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

#Clean up variable & value labels
def clean_label(x):
    #Remove HTML
    clean_x = strip_tags(x)
    #Remove line breaks
    clean_x = re.sub(r'\n', ' ', clean_x)
    #Remove extra white space (2+)
    clean_x = re.sub(r'\s{2,}', ' ', clean_x)
    #then remove leading white space
    clean_x  = re.sub(r'^\s','', clean_x)
    #finally remove trailing white space
    clean_x = re.sub(r'\s$','', clean_x)
    return clean_x
    
data_dict_df['Var_Label'] = [clean_label(i) for i in data_dict_df['Var_Label'].astype(str)]
data_dict_df['Value_Label'] = [clean_label(i) for i in data_dict_df['Value_Label'].astype(str)]

## Step 4: Save as csv <a class="anchor" id="4"></a>

In [ ]:
#Export data_dict_df
data_dict_df.to_csv('data_dictionary.csv', index = False)